### Intro

This notebook continues to explore gradient calculation and backpropagation, this time experimenting with developing a prototype of an autograd engine. The main goal is to improve an understanding about the process automatic differentiation for computational graphs.

A computational graph object will be constructed along with `backprop` method.

Plan of attack:

`ComputeNode` object which will represent a node in the DAG
`Operation` objects representing different operations between the inputs, such as addition, multiplication etc. Those has to be manually defined to compute output value of two inputs, and partial derivative in respect to some input value.
`ComputationalGraph` object - a container for the DAG, providing access to the multi-node root and the single-node leaf node. It must have a way to:
- propagate forward (`forward_pass`): compute values of each node in respect to its inputs
- and backwards (`backprop`): compute gradients of each node


In [23]:
import numpy as np
from typing import List

### Attempting autograd

In [127]:
class ComputeNode():
    def __init__(self, left_input=None, right_input=None, value=None, operation=None):
        """
        operation=None means it is a head node of the graph
        
        """
        if left_input is None and right_input is None and value is None:
            raise ValueError('Value is N/A and operation is N/A - error in setup of the Node')
        self.left_input = left_input
        self.right_input = right_input
        if left_input is not None and right_input is not None:
            self.left_input.next = self
            self.right_input.next = self
        self.operation = operation
        self.value = value
        # This will be changed during init
        self.next = None
        self.gradient = None


    def __repr__(self):
        if self.operation:
            return f'ComputeNode: [{self.operation.symbol}]'
        else:
            return f'ComputeNode: [{self.value}]'

    def compute_value(self):
        if self.value is not None:
            print('Value is alrady calculated for this node')
        elif self.operation:
            self.value = self.operation.compute(self.left_input, self.right_input)
        else:
            raise ValueError('Value is N/A and operation is N/A - error in setup of the Node')

    def get_value(self):
        if self.value is None:
            self.compute_value()
        return self.value

    def compute_gradient(self):
        # Gradient of the final node is equal to 1
        if self.next is None:
            self.gradient = 1
        else:
            op = self.next.operation
            # getting another node
            other = self.next.get_other_input(self)
            # application of chain rule: df/dx = df/dg * dg/dx
            self.gradient = op.partial_derivative(other) * self.next.get_gradient()

    def get_gradient(self):
        if self.gradient is None:
            self.compute_gradient()
        return self.gradient

    def get_other_input(self, node):
        if self.left_input is not None and self.right_input is not None:
            if node == self.left_input:
                return self.right_input
            else:
                return self.left_input
        else:
            raise ValueError()



In [82]:
class AdditionOperation():
    symbol = '+'
    def compute(left: ComputeNode, right: ComputeNode) -> float:
        out = left.get_value() + right.get_value()
        return out

    def partial_derivative(other_node: ComputeNode):
        """
        other_node - the 'other' input node in the current computational
        node, which gradient we want to compute
        """
        return 1

class MultiplicationOperation():
    symbol = '*'
    def compute(left: ComputeNode, right: ComputeNode) -> float:
        out = left.get_value() * right.get_value()
        return out

    def partial_derivative(other_node: ComputeNode):
        """
        other_node - the 'other' input node in the current computational
        node, which gradient we want to compute
        """
        return other_node.value

In [116]:
class ComputationalGraph:
    def __init__(self, root: List[ComputeNode], leaf: ComputeNode):
        # root can be one or several nodes
        self.root = root
        self.leaf = leaf


    def forward_pass(self):
        """
        Calculates forward pass of the computational graph,
        i.e. get value of the function.
        """
        return self.leaf.get_value()

    def backprop(self):
        """
        Get gradient vector in respect to the inputs
        through backpropagation
        """
        self.forward_pass()
        out = [n.get_gradient() for n in self.root]
        return out
        
        


### A simple test

Backprop will be performed on the same scalar-valued function `f(x, y, z) = (x + y) * z` at a point [-2, 5, -4].

Expected answer: [-4, -4, 3] (see previous notebook for computations).

In [128]:
# graph will be constructed manually
x = ComputeNode(value=-2)
print(x)
y = ComputeNode(value=5)
print(y)
z = ComputeNode(value=-4)
print(z)
q = ComputeNode(left_input=x, right_input=y, operation=AdditionOperation)
print(q)
f = ComputeNode(left_input=q, right_input=z, operation=MultiplicationOperation)
print(f)
g = ComputationalGraph(root=[x, y, z], leaf=f)

ComputeNode: [-2]
ComputeNode: [5]
ComputeNode: [-4]
ComputeNode: [+]
ComputeNode: [*]


In [130]:
results = g.backprop()

# Print the results
print("df/dx =", results[0])
print("df/dy =", results[1])
print("df/dz =", results[2])

df/dx = -4
df/dy = -4
df/dz = 3


### Conclusion

This small proof of concept shows that it is not extremely complicated to create a representation for a computational graph and unwraps computational procedures in backpropagation.

Next steps:

- Extend number of operations and run this on a real loss function
- Move on towards representation of vectorised operations (so far operating only on scalars)